<a href="https://colab.research.google.com/github/uday1205/Team_Nexus_Climate_change_dashboard/blob/main/Copy_of_Untitled2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask pyngrok matplotlib pandas

In [ ]:
from flask import Flask, render_template_string, request, jsonify
from pyngrok import ngrok
import matplotlib.pyplot as plt
import pandas as pd
import io
import base64

In [ ]:
app = Flask(__name__)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
file_path = '/content/gdrive/MyDrive/python/climate_change_indicators.csv'

# Load data into a pandas DataFrame
df = pd.read_csv(file_path)


In [ ]:
!ngrok authtoken 2pevURo0F82PRQgvKPqblufURRE_2Z1eQgdzdpa5cajvTH7mA


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
def start_ngrok():
    public_url = ngrok.connect(5000)
    print(f"Public URL: {public_url}")
    return public_url

In [ ]:
from flask import Flask, render_template_string, request, jsonify
import pandas as pd
import matplotlib.pyplot as plt
import io
import base64

app = Flask(__name__)


@app.route('/')
def home():
    # Get the list of years from 1960 to 2000 (assuming the columns are in the format F1960, F1961, ..., F2000)
    years = [str(year) for year in range(1961, 2023)]  # 1961 to 2023
    countries = df['Country'].unique().tolist()

    # HTML content for the dashboard
    html_content = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Climate Change Dashboard</title>
        <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/bootstrap/5.3.0/css/bootstrap.min.css">
        <script src="https://cdnjs.cloudflare.com/ajax/libs/jquery/3.6.0/jquery.min.js"></script>
    </head>
    <body style="background-image: url('/content/gdrive/MyDrive/python/image123.jpeg'); background-size: cover; background-position: center; height: 100vh; padding: 100px; border-radius: 10px;">
        <div class="container mt-5">
            <h1 class="text-center">Climate Change Dashboard</h1>
            <div class="row my-4">
                <div class="col-md-6">
                    <label for="startYearSelect" class="form-label">Select Start Year:</label>
                    <select id="startYearSelect" class="form-select">
                        {% for year in years %}
                            <option value="{{ year }}">{{ year }}</option>
                        {% endfor %}
                    </select>
                </div>
                <div class="col-md-6">
                    <label for="endYearSelect" class="form-label">Select End Year:</label>
                    <select id="endYearSelect" class="form-select">
                        {% for year in years %}
                            <option value="{{ year }}">{{ year }}</option>
                        {% endfor %}
                    </select>
                </div>
            </div>
            <div class="row my-4">
                <div class="col-md-6">
                    <label for="countrySelect" class="form-label">Select Country:</label>
                    <select id="countrySelect" class="form-select">
                        {% for country in countries %}
                            <option value="{{ country }}">{{ country }}</option>
                        {% endfor %}
                    </select>
                </div>
            </div>
            <button id="generatePlot" class="btn btn-primary">Generate Plot</button>
            <div id="chart" class="mt-5 text-center">
                <!-- Plot will be displayed here -->
            </div>
        </div>

        <script>
            $(document).ready(function () {
                // Handle plot generation on button click
                $('#generatePlot').click(function () {
                    const selectedStartYear = $('#startYearSelect').val();
                    const selectedEndYear = $('#endYearSelect').val();
                    const selectedCountry = $('#countrySelect').val();

                    // AJAX request to generate plot
                    $.post('/generate_plot', { start_year: selectedStartYear, end_year: selectedEndYear, country: selectedCountry }, function (data) {
                        $('#chart').html(`<img src="data:image/png;base64,${data.plot_data}" alt="Temperature Plot" class="img-fluid">`);
                    }).fail(function (error) {
                        alert(error.responseJSON.error);
                    });
                });
            });
        </script>
    </body>
    </html>
    """

    return render_template_string(html_content, years=years, countries=countries)


@app.route('/generate_plot', methods=['POST'])
def generate_plot():
    # Get the start year, end year, and country from the request
    start_year = int(request.form['start_year'])
    end_year = int(request.form['end_year'])
    selected_country = request.form['country']

    # Filter the DataFrame based on the selected country
    country_data = df[df['Country'] == selected_country]

    # Extract temperature changes for the selected country within the selected year range
    years = [str(year) for year in range(start_year, end_year + 1)]  # Get years in the selected range
    temperature_changes = [country_data[f'F{year}'].values[0] for year in years if f'F{year}' in country_data.columns]

    # Check if we have valid data for the selected range
    if len(temperature_changes) == 0:
        return jsonify({'error': 'No data available for the selected range and country'}), 400

    # Plot the data for the selected country and year range
    plt.figure(figsize=(10, 6))
    plt.plot(years, temperature_changes, marker='o', linestyle='-', color='b')
    plt.title(f'Temperature Change for {selected_country} ({start_year}-{end_year})')
    plt.xlabel('Year')
    plt.ylabel('Temperature Change (°C)')
    plt.xticks(rotation=45)

    # Save the plot to a PNG image in memory
    img = io.BytesIO()
    plt.savefig(img, format='png')
    img.seek(0)
    plot_data = base64.b64encode(img.read()).decode('utf-8')
    plt.close()

    # Return the plot as base64-encoded image data
    return jsonify({'plot_data': plot_data})





In [ ]:
if __name__ == "__main__":
    start_ngrok()  # Start ngrok to expose Flask app
    app.run()

Public URL: NgrokTunnel: "https://d890-104-199-189-200.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
from flask import Flask, render_template_string, request, jsonify
import pandas as pd
import matplotlib.pyplot as plt
import io
import base64

app = Flask(__name__)


@app.route('/')
def home():
    years = [str(year) for year in range(1961, 2023)]  # Adjust years as needed
    countries = df['Country'].unique().tolist()

    # HTML content for the dashboard
    html_content = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Climate Change Dashboard</title>
        <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/bootstrap/5.3.0/css/bootstrap.min.css">
        <script src="https://cdnjs.cloudflare.com/ajax/libs/jquery/3.6.0/jquery.min.js"></script>
    </head>
    <body style="background-image: url('your_image_path_here.png'); background-size: cover; background-position: center; height: 100vh; padding: 100px; border-radius: 10px;">
        <div class="container mt-5">
            <h1 class="text-center">Climate Change Dashboard</h1>

            <!-- Select start and end year -->
            <div class="row my-4">
                <div class="col-md-6">
                    <label for="startYearSelect" class="form-label">Select Start Year:</label>
                    <select id="startYearSelect" class="form-select">
                        {% for year in years %}
                            <option value="{{ year }}">{{ year }}</option>
                        {% endfor %}
                    </select>
                </div>
                <div class="col-md-6">
                    <label for="endYearSelect" class="form-label">Select End Year:</label>
                    <select id="endYearSelect" class="form-select">
                        {% for year in years %}
                            <option value="{{ year }}">{{ year }}</option>
                        {% endfor %}
                    </select>
                </div>
            </div>
           <script>
                 $(document).ready(function () {
                     // Handle plot generation on button click
                    $('#generatePlot').click(function () {
                    const selectedStartYear = $('#startYearSelect').val();
                    const selectedEndYear = $('#endYearSelect').val();
                    const selectedCountry = $('#countrySelect').val();

                    // AJAX request to generate plot
                    $.post('/generate_plot', { start_year: selectedStartYear, end_year: selectedEndYear, country: selectedCountry }, function (data) {
                        $('#chart').html(`<img src="data:image/png;base64,${data.plot_data}" alt="Temperature Plot" class="img-fluid">`);
                    }).fail(function (error) {
                        alert(error.responseJSON.error);
                    });
                });
            });
        </script>
            <!-- Select countries -->
            <div class="row my-4">
                <div class="col-md-6">
                    <label for="countrySelect" class="form-label">Select Countries:</label>
                    <select id="countrySelect" class="form-select">
                        {% for country in countries %}
                            <option value="{{ country }}">{{ country }}</option>
                        {% endfor %}
                    </select>
                </div>
            </div>

            <!-- Button to generate plot -->
            <button id="generatePlot" class="btn btn-primary">Generate Temperature Plot</button>

            <div id="chart" class="mt-5 text-center">
                <!-- Plot will be displayed here -->
            </div>

            <!-- Select year for top 10 hottest countries -->
            <div class="row my-4">
                <div class="col-md-6">
                    <label for="yearSelect" class="form-label">Select Year for Top 10 Hottest Countries:</label>
                    <select id="yearSelect" class="form-select">
                        {% for year in years %}
                            <option value="{{ year }}">{{ year }}</option>
                        {% endfor %}
                    </select>
                </div>
            </div>

            <!-- Button to generate top 10 chart -->
            <button id="generateTop10Chart" class="btn btn-success">Generate Top 10 Hottest Countries Chart</button>

            <div id="top10Chart" class="mt-5 text-center">
                <!-- Top 10 Hottest Countries chart will be displayed here -->
            </div>
        </div>

        <script>
            $(document).ready(function () {
                // Handle temperature change plot generation
                $('#generatePlot').click(function () {
                    const selectedStartYear = $('#startYearSelect').val();
                    const selectedEndYear = $('#endYearSelect').val();
                    const selectedCountries = $('#countrySelect').val();  // Get selected countries

                    // Send data to backend
                    $.post('/generate_plot', {
                        start_year: selectedStartYear,
                        end_year: selectedEndYear,
                        countries: selectedCountries
                    }, function (data) {
                        if (data.plot_data) {
                            $('#chart').html(`<img src="data:image/png;base64,${data.plot_data}" alt="Temperature Plot" class="img-fluid">`);
                        } else {
                            alert('Error: ' + data.error);
                        }
                    }).fail(function (error) {
                        alert('Error: ' + error.responseJSON.error);
                    });
                });

                // Handle Top 10 hottest countries chart generation
                $('#generateTop10Chart').click(function () {
                    const selectedYear = $('#yearSelect').val();

                    // Send request to backend
                    $.post('/generate_top10', { year: selectedYear }, function (data) {
                        if (data.plot_data) {
                            $('#top10Chart').html(`<img src="data:image/png;base64,${data.plot_data}" alt="Top 10 Hottest Countries Chart" class="img-fluid">`);
                        } else {
                            alert('Error: ' + data.error);
                        }
                    }).fail(function (error) {
                        alert('Error: ' + error.responseJSON.error);
                    });
                });
            });
        </script>
    </body>
    </html>
    """

    return render_template_string(html_content, years=years, countries=countries)


@app.route('/generate_plot', methods=['POST'])
def generate_plot():
    # Get the start year, end year, and country from the request
    start_year = int(request.form['start_year'])
    end_year = int(request.form['end_year'])
    selected_country = request.form['country']

    # Filter the DataFrame based on the selected country
    country_data = df[df['Country'] == selected_country]

    # Extract temperature changes for the selected country within the selected year range
    years = [str(year) for year in range(start_year, end_year + 1)]  # Get years in the selected range
    temperature_changes = [country_data[f'F{year}'].values[0] for year in years if f'F{year}' in country_data.columns]

    # Check if we have valid data for the selected range
    if len(temperature_changes) == 0:
        return jsonify({'error': 'No data available for the selected range and country'}), 400

    # Plot the data for the selected country and year range
    plt.figure(figsize=(10, 6))
    plt.plot(years, temperature_changes, marker='o', linestyle='-', color='b')
    plt.title(f'Temperature Change for {selected_country} ({start_year}-{end_year})')
    plt.xlabel('Year')
    plt.ylabel('Temperature Change (°C)')
    plt.xticks(rotation=45)

    # Save the plot to a PNG image in memory
    img = io.BytesIO()
    plt.savefig(img, format='png')
    img.seek(0)
    plot_data = base64.b64encode(img.read()).decode('utf-8')
    plt.close()

    # Return the plot as base64-encoded image data
    return jsonify({'plot_data': plot_data})




@app.route('/generate_top10', methods=['POST'])
def generate_top10():
    try:
        # Get selected year from request
        year = int(request.form['year'])

        # Ensure the year exists in the dataset
        if f'F{year}' not in df.columns:
            return jsonify({'error': f'No data available for the year {year}'}), 400

        # Calculate average temperature change for each country
        df_year = df[['Country', f'F{year}']].dropna()  # Drop countries with missing data
        df_year['Avg_Temperature_Change'] = df_year[f'F{year}']

        # Sort and get the top 10 hottest countries
        top_10 = df_year.sort_values('Avg_Temperature_Change', ascending=False).head(10)

        # Create the bar chart
        plt.figure(figsize=(10, 6))
        plt.bar(top_10['Country'], top_10['Avg_Temperature_Change'], color='tomato')
        plt.title(f'Top 10 Hottest Countries in {year}')
        plt.xlabel('Country')
        plt.ylabel('Temperature Change (°C)')

        # Save the plot to PNG image in memory
        img = io.BytesIO()
        plt.savefig(img, format='png')
        img.seek(0)
        plot_data = base64.b64encode(img.read()).decode('utf-8')
        plt.close()

        # Return the chart as base64-encoded image
        return jsonify({'plot_data': plot_data})

    except Exception as e:
        return jsonify({'error': 'An error occurred while generating the chart'}), 500




In [ ]:
if __name__ == "__main__":
    start_ngrok()  # Start ngrok to expose Flask app
    app.run()

Public URL: NgrokTunnel: "https://f571-35-202-86-253.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [19/Dec/2024 13:06:28] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Dec/2024 13:06:28] "GET /your_image_path_here.png HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [19/Dec/2024 13:06:29] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [19/Dec/2024 13:06:34] "POST /generate_plot HTTP/1.1" 400 -
INFO:werkzeug:127.0.0.1 - - [19/Dec/2024 13:06:34] "POST /generate_plot HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Dec/2024 13:06:44] "POST /generate_top10 HTTP/1.1" 200 -
